# Generative Adversarial Network

In this notebook, we'll be building a generative adversarial network (GAN) trained on the network flow dataset.

## Import Libraries

In [1]:
from __future__ import absolute_import, division, print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from importlib import reload
import os
import time
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

/scratch/sj2363/hsn/attack_generate/gan_attack_generate/.env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow version: 1.8.0
Eager execution: True


In [2]:
import utils, models
reload(utils)
reload(models)
from models import Generator, Discriminator
from utils import max_norm, parse_feature_label, train_one_epoch

## Define Constants

In [20]:
# 输出的summary的保存目录:不要改
OUTPUT_DIR = 'SUMMARY/'

# 输出的模型的保存目录:不要改
CHECKPOINT_DIR = 'CHECKPOINT/'

# 保存data的目录:不要改
DATA_DIR = 'data'

# 相关feature:不太需要改应该
RELEVANT_FEATURES = [' Source Port', ' Destination Port', ' Flow Duration', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', 'Bwd Packet Length Max', ' Bwd Packet Length Min', 'Flow Bytes/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Fwd Header Length', ' Bwd Packets/s', ' Packet Length Mean', ' ACK Flag Count', ' Down/Up Ratio', ' Avg Fwd Segment Size', ' Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk', ' Bwd Avg Bytes/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' act_data_pkt_fwd', ' Active Std', ' Active Min', ' Idle Max']

# Label的名字:不太需要改应该
LABEL_NAME = ' Label'

# 记录summary的频率:不太需要改应该
LOG_INTERVAL = 10

# 数据集路径
IDS_DATASET = os.path.join('data', 'ids2017_sampled.csv')

# benign flow的label
BENIGN_LABEL = 0

# attack flow的label
ATTACK_LABEL = 2

# training dataset占总dataset的比例
TRAIN_FRAC = 0.3

# 要修改的feature的数量
FEATURE_NUM_MODIFIED = 15

# learning rate
LEARNING_RATE = 0.01

# 训练的epochs数
EPOCHS = 20000

# 是否随机选择feature
RANDOM_SELECT_FEATURE=False

## 提示：
## 训练结束后，把SUMMARY和CHECKPOINT目录重命名后保存

## Load and normalize data

In [4]:
# quick view of dataset
!head -n5 {IDS_DATASET}

Flow ID, Source IP, Source Port, Destination IP, Destination Port, Protocol, Timestamp, Flow Duration, Total Fwd Packets, Total Backward Packets,Total Length of Fwd Packets, Total Length of Bwd Packets, Fwd Packet Length Max, Fwd Packet Length Min, Fwd Packet Length Mean, Fwd Packet Length Std,Bwd Packet Length Max, Bwd Packet Length Min, Bwd Packet Length Mean, Bwd Packet Length Std,Flow Bytes/s, Flow Packets/s, Flow IAT Mean, Flow IAT Std, Flow IAT Max, Flow IAT Min,Fwd IAT Total, Fwd IAT Mean, Fwd IAT Std, Fwd IAT Max, Fwd IAT Min,Bwd IAT Total, Bwd IAT Mean, Bwd IAT Std, Bwd IAT Max, Bwd IAT Min,Fwd PSH Flags, Bwd PSH Flags, Fwd URG Flags, Bwd URG Flags, Fwd Header Length, Bwd Header Length,Fwd Packets/s, Bwd Packets/s, Min Packet Length, Max Packet Length, Packet Length Mean, Packet Length Std, Packet Length Variance,FIN Flag Count, SYN Flag Count, RST Flag Count, PSH Flag Count, ACK Flag Count, URG Flag Count, CWE Flag Count, ECE Flag Count, Down/Up Ratio, Average Packet Size, Av

In [5]:
# read csv
df = pd.read_csv(IDS_DATASET)

In [6]:
# extract relevant features and label name
df = df[RELEVANT_FEATURES + [LABEL_NAME]]

In [7]:
# extract bengin and attack flows we want
benign_df, attack_df = df[(df[LABEL_NAME] == BENIGN_LABEL)], df[(df[LABEL_NAME] == ATTACK_LABEL)]

In [8]:
# rewrite label values
benign_df.loc[:, LABEL_NAME] = 0
attack_df.loc[:, LABEL_NAME] = 1

/scratch/sj2363/hsn/attack_generate/gan_attack_generate/.env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
# convert to numpy
benign, attack = benign_df.values, attack_df.values

In [10]:
# max normalization
benign[:, :len(RELEVANT_FEATURES)] = max_norm(benign[:, :len(RELEVANT_FEATURES)])
attack[:, :len(RELEVANT_FEATURES)] = max_norm(attack[:, :len(RELEVANT_FEATURES)])

In [11]:
# do train, test split separately on benign and attack
benign_train, benign_test = train_test_split(benign, train_size=TRAIN_FRAC)
attack_train, attack_test = train_test_split(attack, train_size=TRAIN_FRAC)

/scratch/sj2363/hsn/attack_generate/gan_attack_generate/.env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [12]:
# concat to get testing data
test_np = np.concatenate([benign_test, attack_test])

## Generate Dataset

In [13]:
# convert to benign train dataset and attack train dataset
benign_train_dataset, attack_train_dataset = tf.data.Dataset.from_tensor_slices(benign_train), tf.data.Dataset.from_tensor_slices(attack_train)

In [14]:
# deal with benign train dataset
benign_train_dataset = benign_train_dataset.map(parse_feature_label)
benign_train_dataset = benign_train_dataset.shuffle(buffer_size=benign_train.shape[0] * 5)  # randomize
benign_train_dataset = benign_train_dataset.batch(100) # make batch

# View a single example entry from a batch
benign_features, benign_label = iter(benign_train_dataset).next()
print("benign example features:", benign_features[0])
print("benign example label:", benign_label[0])

benign example features: tf.Tensor(
[9.47178047e-01 8.26536500e-04 5.01436585e-04 3.35219766e-04
 1.19429590e-06 5.73630137e-03 1.15318417e-01 1.19541447e-02
 9.38611530e-04 0.00000000e+00 5.09872877e-04 9.38611530e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.50946146e-05
 1.66212353e-05 4.26122037e-02 0.00000000e+00 1.42857143e-01
 2.19922380e-02 8.50946146e-05 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.35219766e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00], shape=(41,), dtype=float64)
benign example label: tf.Tensor(0, shape=(), dtype=int32)


In [15]:
# deal with attack train dataset
attack_train_dataset = attack_train_dataset.map(parse_feature_label)
attack_train_dataset = attack_train_dataset.shuffle(buffer_size=attack_train.shape[0] * 5)  # randomize
attack_train_dataset = attack_train_dataset.batch(100) # make batch

# View a single example entry from a batch
attack_features, attack_label = iter(attack_train_dataset).next()
print("attack example features:", attack_features[0])
print("attack example label:", attack_label[0])

attack example features: tf.Tensor(
[2.93579932e-01 0.00000000e+00 2.52223495e-08 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 8.92857154e-08 0.00000000e+00 2.54237290e-08 2.75573192e-04
 2.52100842e-08 8.90585253e-08 0.00000000e+00 1.53668844e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.76923077e-02
 0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
 0.00000000e+00 5.76923077e-02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 8.59589041e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00], shape=(41,), dtype=float64)
attack example label: tf.Tensor(1, shape=(), dtype=int32)


## Build Model

In [16]:
generator = Generator(input_shape=len(RELEVANT_FEATURES), output_shape=FEATURE_NUM_MODIFIED)
discriminator = Discriminator()

## Build Optimizer

In [17]:
generator_optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
discriminator_optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)

## Build Tensorflow Training Environment

In [18]:
step_counter = tf.train.get_or_create_global_step()
summary_writer = tf.contrib.summary.create_file_writer(
      OUTPUT_DIR, flush_millis=1000)
checkpoint_prefix = os.path.join(CHECKPOINT_DIR, 'ckpt')
latest_cpkt = tf.train.latest_checkpoint(CHECKPOINT_DIR)
if latest_cpkt:
    print('Using latest checkpoint at ' + latest_cpkt)
model_objects = {
    'generator': generator,
    'discriminator': discriminator,
    'generator_optimizer': generator_optimizer,
    'discriminator_optimizer': discriminator_optimizer,
    'step_counter': step_counter
}
checkpoint = tfe.Checkpoint(**model_objects)
# Restore variables on creation if a checkpoint exists.
checkpoint.restore(latest_cpkt)

## Train

In [19]:
with tf.device('/gpu:0'):
    train_start = time.time()
    for _ in range(EPOCHS):
        start = time.time()
        with summary_writer.as_default():
            train_one_epoch(benign_dataset=benign_train_dataset,
                        attack_dataset=attack_train_dataset,
                        log_interval=LOG_INTERVAL,
                        modified_feature_num=FEATURE_NUM_MODIFIED,
                        random_select_feature=RANDOM_SELECT_FEATURE,
                        **model_objects)
        end = time.time()
        checkpoint.save(checkpoint_prefix)
        print('\nTrain time for epoch #%d (step %d): %f' %
            (checkpoint.save_counter.numpy(),
             checkpoint.step_counter.numpy(),
             end - start))
    print('\nTotal training time for {epoch} epoch(s) is {second}'.format(
        second=time.time() - train_start,
        epoch=EPOCHS
    ))

Batch #10	Average Generator Loss: 2.378508	Average Discriminator Loss: 0.982748

Train time for epoch #1 (step 1): 1.606882
Batch #10	Average Generator Loss: 4.524645	Average Discriminator Loss: 1.633949

Train time for epoch #2 (step 2): 1.416448
Batch #10	Average Generator Loss: 7.508555	Average Discriminator Loss: 1.804198

Train time for epoch #3 (step 3): 1.417856
Batch #10	Average Generator Loss: 3.670270	Average Discriminator Loss: 1.380155

Train time for epoch #4 (step 4): 1.375160
Batch #10	Average Generator Loss: 6.219086	Average Discriminator Loss: 0.862147

Train time for epoch #5 (step 5): 1.416094
Batch #10	Average Generator Loss: 3.885069	Average Discriminator Loss: 0.913086

Train time for epoch #6 (step 6): 1.397432
Batch #10	Average Generator Loss: 6.967915	Average Discriminator Loss: 0.540294

Train time for epoch #7 (step 7): 1.409770
Batch #10	Average Generator Loss: 11.832751	Average Discriminator Loss: 0.347888

Train time for epoch #8 (step 8): 1.390265
Batch #

Batch #10	Average Generator Loss: 43.719670	Average Discriminator Loss: 0.060091

Train time for epoch #66 (step 66): 1.383076
Batch #10	Average Generator Loss: 48.472795	Average Discriminator Loss: 0.167133

Train time for epoch #67 (step 67): 1.305131
Batch #10	Average Generator Loss: 41.060191	Average Discriminator Loss: 0.102658

Train time for epoch #68 (step 68): 1.405823
Batch #10	Average Generator Loss: 53.857782	Average Discriminator Loss: 0.115115

Train time for epoch #69 (step 69): 1.339426
Batch #10	Average Generator Loss: 42.412526	Average Discriminator Loss: 0.044617

Train time for epoch #70 (step 70): 1.389057
Batch #10	Average Generator Loss: 45.034400	Average Discriminator Loss: 0.241916

Train time for epoch #71 (step 71): 1.552267
Batch #10	Average Generator Loss: 47.179978	Average Discriminator Loss: 0.044026

Train time for epoch #72 (step 72): 1.424548
Batch #10	Average Generator Loss: 53.258494	Average Discriminator Loss: 0.166915

Train time for epoch #73 (ste


Train time for epoch #130 (step 130): 1.444254
Batch #10	Average Generator Loss: 79.783440	Average Discriminator Loss: 0.019578

Train time for epoch #131 (step 131): 1.428519
Batch #10	Average Generator Loss: 75.057911	Average Discriminator Loss: 0.011092

Train time for epoch #132 (step 132): 1.349677
Batch #10	Average Generator Loss: 82.271469	Average Discriminator Loss: 0.007619

Train time for epoch #133 (step 133): 1.403341
Batch #10	Average Generator Loss: 78.897205	Average Discriminator Loss: 0.008742

Train time for epoch #134 (step 134): 1.451640
Batch #10	Average Generator Loss: 74.785960	Average Discriminator Loss: 0.039000

Train time for epoch #135 (step 135): 1.428718
Batch #10	Average Generator Loss: 73.217596	Average Discriminator Loss: 0.093775

Train time for epoch #136 (step 136): 1.419865
Batch #10	Average Generator Loss: 71.006862	Average Discriminator Loss: 0.031512

Train time for epoch #137 (step 137): 1.345772
Batch #10	Average Generator Loss: 75.677615	Avera

Batch #10	Average Generator Loss: 93.416564	Average Discriminator Loss: 0.000964

Train time for epoch #194 (step 194): 1.358894
Batch #10	Average Generator Loss: 100.762606	Average Discriminator Loss: 0.001708

Train time for epoch #195 (step 195): 1.406810
Batch #10	Average Generator Loss: 99.112370	Average Discriminator Loss: 0.001216

Train time for epoch #196 (step 196): 1.387016
Batch #10	Average Generator Loss: 108.202218	Average Discriminator Loss: 0.022989

Train time for epoch #197 (step 197): 1.342002
Batch #10	Average Generator Loss: 101.378024	Average Discriminator Loss: 0.000978

Train time for epoch #198 (step 198): 1.458368
Batch #10	Average Generator Loss: 100.538174	Average Discriminator Loss: 0.001484

Train time for epoch #199 (step 199): 1.472627
Batch #10	Average Generator Loss: 129.895750	Average Discriminator Loss: 0.118197

Train time for epoch #200 (step 200): 1.349593
Batch #10	Average Generator Loss: 92.995124	Average Discriminator Loss: 0.175635

Train time


Train time for epoch #257 (step 257): 1.308382
Batch #10	Average Generator Loss: 130.791785	Average Discriminator Loss: 0.021970

Train time for epoch #258 (step 258): 1.543749
Batch #10	Average Generator Loss: 115.742498	Average Discriminator Loss: 0.013297

Train time for epoch #259 (step 259): 1.534768
Batch #10	Average Generator Loss: 116.794262	Average Discriminator Loss: 0.044320

Train time for epoch #260 (step 260): 1.393329
Batch #10	Average Generator Loss: 138.601792	Average Discriminator Loss: 0.003807

Train time for epoch #261 (step 261): 1.360785
Batch #10	Average Generator Loss: 101.711391	Average Discriminator Loss: 0.011479

Train time for epoch #262 (step 262): 1.348392
Batch #10	Average Generator Loss: 118.051826	Average Discriminator Loss: 0.007049

Train time for epoch #263 (step 263): 1.432537
Batch #10	Average Generator Loss: 117.328975	Average Discriminator Loss: 0.001182

Train time for epoch #264 (step 264): 1.400729
Batch #10	Average Generator Loss: 117.2260

...

Batch #10	Average Generator Loss: 3114.286646	Average Discriminator Loss: 0.000000

Train time for epoch #19739 (step 19739): 1.401070
Batch #10	Average Generator Loss: 3161.310046	Average Discriminator Loss: 0.000000

Train time for epoch #19740 (step 19740): 1.399010
Batch #10	Average Generator Loss: 3417.256311	Average Discriminator Loss: 0.000000

Train time for epoch #19741 (step 19741): 1.353657
Batch #10	Average Generator Loss: 3261.328516	Average Discriminator Loss: 0.000000

Train time for epoch #19742 (step 19742): 2.095576
Batch #10	Average Generator Loss: 3297.294006	Average Discriminator Loss: 0.000000

Train time for epoch #19743 (step 19743): 1.341737
Batch #10	Average Generator Loss: 3228.036682	Average Discriminator Loss: 0.000000

Train time for epoch #19744 (step 19744): 1.284030
Batch #10	Average Generator Loss: 3510.142126	Average Discriminator Loss: 0.000000

Train time for epoch #19745 (step 19745): 1.300472
Batch #10	Average Generator Loss: 3346.410315	Average D

Batch #10	Average Generator Loss: 2885.774207	Average Discriminator Loss: 0.000000

Train time for epoch #19800 (step 19800): 2.067914
Batch #10	Average Generator Loss: 2615.873596	Average Discriminator Loss: 0.000000

Train time for epoch #19801 (step 19801): 1.291679
Batch #10	Average Generator Loss: 2489.018701	Average Discriminator Loss: 0.224342

Train time for epoch #19802 (step 19802): 1.488900
Batch #10	Average Generator Loss: 2445.191345	Average Discriminator Loss: 0.000000

Train time for epoch #19803 (step 19803): 1.331372
Batch #10	Average Generator Loss: 2067.184857	Average Discriminator Loss: 0.644779

Train time for epoch #19804 (step 19804): 1.931381
Batch #10	Average Generator Loss: 2223.415891	Average Discriminator Loss: 0.018753

Train time for epoch #19805 (step 19805): 1.417081
Batch #10	Average Generator Loss: 2472.286023	Average Discriminator Loss: 0.003393

Train time for epoch #19806 (step 19806): 1.318980
Batch #10	Average Generator Loss: 3136.589941	Average D

Batch #10	Average Generator Loss: 3238.066931	Average Discriminator Loss: 0.000000

Train time for epoch #19861 (step 19861): 1.329679
Batch #10	Average Generator Loss: 3338.629590	Average Discriminator Loss: 0.000001

Train time for epoch #19862 (step 19862): 1.926845
Batch #10	Average Generator Loss: 3224.367651	Average Discriminator Loss: 0.000000

Train time for epoch #19863 (step 19863): 1.289682
Batch #10	Average Generator Loss: 3510.868091	Average Discriminator Loss: 0.000198

Train time for epoch #19864 (step 19864): 1.487014
Batch #10	Average Generator Loss: 2964.886169	Average Discriminator Loss: 0.000005

Train time for epoch #19865 (step 19865): 1.334307
Batch #10	Average Generator Loss: 3543.270557	Average Discriminator Loss: 0.000015

Train time for epoch #19866 (step 19866): 2.015849
Batch #10	Average Generator Loss: 3424.291528	Average Discriminator Loss: 0.000013

Train time for epoch #19867 (step 19867): 1.330038
Batch #10	Average Generator Loss: 2710.016760	Average D

Batch #10	Average Generator Loss: 2865.870264	Average Discriminator Loss: 0.000000

Train time for epoch #19922 (step 19922): 1.385382
Batch #10	Average Generator Loss: 2506.170837	Average Discriminator Loss: 0.000000

Train time for epoch #19923 (step 19923): 1.450249
Batch #10	Average Generator Loss: 2312.898376	Average Discriminator Loss: 0.000000

Train time for epoch #19924 (step 19924): 1.297354
Batch #10	Average Generator Loss: 2516.822986	Average Discriminator Loss: 0.000000

Train time for epoch #19925 (step 19925): 1.892974
Batch #10	Average Generator Loss: 2166.796191	Average Discriminator Loss: 0.000000

Train time for epoch #19926 (step 19926): 1.346081
Batch #10	Average Generator Loss: 2249.169971	Average Discriminator Loss: 0.000000

Train time for epoch #19927 (step 19927): 1.426363
Batch #10	Average Generator Loss: 3022.997314	Average Discriminator Loss: 0.000000

Train time for epoch #19928 (step 19928): 1.284631
Batch #10	Average Generator Loss: 2856.382751	Average D

Batch #10	Average Generator Loss: 2646.684009	Average Discriminator Loss: 0.000000

Train time for epoch #19983 (step 19983): 1.311570
Batch #10	Average Generator Loss: 2683.956519	Average Discriminator Loss: 0.000000

Train time for epoch #19984 (step 19984): 2.085973
Batch #10	Average Generator Loss: 3231.097961	Average Discriminator Loss: 0.000000

Train time for epoch #19985 (step 19985): 1.392604
Batch #10	Average Generator Loss: 2904.628882	Average Discriminator Loss: 0.000000

Train time for epoch #19986 (step 19986): 1.341294
Batch #10	Average Generator Loss: 2763.535754	Average Discriminator Loss: 0.000000

Train time for epoch #19987 (step 19987): 1.343883
Batch #10	Average Generator Loss: 3470.336011	Average Discriminator Loss: 0.000000

Train time for epoch #19988 (step 19988): 2.099783
Batch #10	Average Generator Loss: 3204.056189	Average Discriminator Loss: 0.003795

Train time for epoch #19989 (step 19989): 1.308348
Batch #10	Average Generator Loss: 2924.782715	Average D

## Save Train/Test Data

In [21]:
np.save(os.path.join(DATA_DIR, 'benign_train.npy'), benign_train)
np.save(os.path.join(DATA_DIR, 'benign_test.npy'), benign_test)
np.save(os.path.join(DATA_DIR, 'attack_train.npy'), attack_train)
np.save(os.path.join(DATA_DIR, 'attack_test.npy'), attack_test)

In [27]:
loaded_benign_train = np.load(os.path.join(DATA_DIR, 'benign_train.npy'))
loaded_benign_test = np.load(os.path.join(DATA_DIR, 'benign_test.npy'))
loaded_attack_train = np.load(os.path.join(DATA_DIR, 'attack_train.npy'))
loaded_attack_test = np.load(os.path.join(DATA_DIR, 'attack_test.npy'))
assert (loaded_benign_train == benign_train).all()
assert (loaded_benign_test == benign_test).all()
assert (loaded_attack_train == attack_train).all()
assert (loaded_attack_test == attack_test).all()

True
True
True
